In [1]:
import warnings

# Filter out specific warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*TDS_COLMETADATA.*")

In [2]:
import os
print('|',"="*10,"Setting the Java Home Env")
os.environ['JAVA_HOME']='/Library/Java/JavaVirtualMachines/jdk-21.jdk/Contents/Home'
os.environ['SPARK_HOME']='/Users/mac/spark'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS']= "notebook"

| ========== Setting the Java Home Env


In [3]:
print('|',"="*10,"Importing libraries")
# import findspark
# import pyspark
# findspark.init()
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession
import os,duckdb,sys
import warnings
from pathlib import Path
# Suppress specific warning
# warnings.filterwarnings("ignore", category=UserWarning, module="pyspark")

| ========== Importing libraries


In [4]:
print('|',"="*10,"Create a spark session")
spark = SparkSession.builder \
                    .appName('Duck to MSSQL') \
                    .config('spark.jars', '/Users/mac/Downloads/sqljdbc_12.4/enu/jars/mssql-jdbc-12.4.2.jre11.jar') \
                    .getOrCreate()


| ========== Create a spark session


23/12/17 14:43:33 WARN Utils: Your hostname, Senyonjos-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.100.37 instead (on interface en0)
23/12/17 14:43:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/12/17 14:43:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/17 14:43:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
spark

In [6]:
print('|',"="*10,"Loading data  from Dudckdb to MSSQL")
print('1.',"Reading stg data from duckdb")
duckdb_path='./engineering.duckdb'
#conneting to duckdb
connection=duckdb.connect(database=str(duckdb_path))
df=connection.sql("SELECT * FROM STG_TRANSACTIONS").to_df()
#convert results to pandas Dataframe
# df=df.fetch_df()S
#convert the result to a pyspark dataframe
df=spark.createDataFrame(df)
connection.close()

#now that we have an RDD , write it to msssql data
properties = {
    "user": "sa",
    "password": "K@roukazeno1",
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver",
    "encrypt": "true",
    "trustServerCertificate": "true"
}

# Correct the JDBC URL
jdbc_url = "jdbc:sqlserver://localhost:1433;databaseName=ENGINEERING"

# df.show()

#readinng data from warehouse 
# acc = spark.read \
#     .jdbc(url=jdbc_url, table="(SELECT TOP 10 * FROM ACCOUNTS)", properties=properties)

try:
    acc = spark.read \
        .jdbc(url=jdbc_url, table="ACCOUNTS", properties=properties)
    # acc.show()
except Exception as e:
    print("An error occurred:", e)
    import traceback
    traceback.print_exc()


# acc.show()

# Loading the data to the warehouse
try:
    df.write \
    .mode('overwrite') \
    .jdbc(url=jdbc_url, table="TRANSACTIONS", properties=properties)
    print('!Transactions Table loaded successfully')
except Exception as e:
    print("Error:", e)
    raise e
finally:
    spark.stop()







| ========== Loading data  from Dudckdb to MSSQL
1. Reading stg data from duckdb


23/12/17 14:43:47 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
23/12/17 14:43:48 WARN TOKEN: ConnectionID:3 ClientConnectionId: bd43be67-a111-4dcb-9b56-4da390acdaf9: SQLServerConnection.supportsTransactions: Discarding unexpected TDS_COLMETADATA (0x81)
23/12/17 14:43:48 WARN TOKEN: ConnectionID:5 ClientConnectionId: 4ff8d047-0e44-4b61-a901-4efacb6c3932: SQLServerConnection.supportsTransactions: Discarding unexpected TDS_COLMETADATA (0x81)
23/12/17 14:43:48 WARN TOKEN: ConnectionID:6 ClientConnectionId: 72b2fd0d-2f30-4d23-9799-588f4c61e3a5: SQLServerConnection.supportsTransactions: Discarding unexpected TDS_COLMETADATA (0x81)
23/12/17 14:43:48 WARN TOKEN: ConnectionID:10 ClientConnectionId: 2fb023c2-667a-4205-853d-bd05d7da4d8a: SQLServerConnection.supportsTransactions: Discarding unex

!Transactions Table loaded successfully
